In [1]:
%reload_ext autoreload
%autoreload 2
import torch
import dagrad.flex as flex
from dagrad.utils import utils
import numpy as np

utils.set_random_seed(1)
torch.manual_seed(1)

No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
def postprocess(B, graph_thres=0.3):
    """Post-process estimated solution:
        (1) Thresholding.
        (2) Remove the edges with smallest absolute weight until a DAG
            is obtained.

    Args:
        B (numpy.ndarray): [d, d] weighted matrix.
        graph_thres (float): Threshold for weighted matrix. Default: 0.3.

    Returns:
        numpy.ndarray: [d, d] weighted matrix of DAG.
    """
    B = np.copy(B)
    B[np.abs(B) <= graph_thres] = 0    # Thresholding
    B, _ = utils.threshold_till_dag(B)

    return B

## Struct Learn via Flex

`flex.struct_learn` is a function that requires the following 5 inputs:
- `dataset`: A data matrix with `n x d` shape, where `n` is the number of samples and `d` is the number of variables/features.

- `model`: The SEM module. We provide standard SEM implementations such as---LinearModel, LogisticModel, MLP

- `constrained_solver`: An instance of a ConstrainedSolver class. We provide implementations such as `PathFollowing` and `AugmentedLagrangian`

- `unconstrained_solver`: An instance of an UnconstrainedSolver class. We provide implementation for `GradientBasedSolver`

- `loss_fn`: Instance of a Loss class. All available losses can be found at flex/loss.py

- `dag_fn`: Instance of a DagFn class. All available DAG functions can be found at flex/dags.py

## Generate Data

In [3]:
# Generate linear data
n, d, s0, graph_type, sem_type = 1000, 20, 20, "ER", "gauss"
linear_B_true = utils.simulate_dag(d, s0, graph_type)
linear_dataset = utils.simulate_linear_sem(linear_B_true, n, sem_type)

# Generate non-linear data
n, d, s0, graph_type, sem_type = 1000, 5, 5, "ER", "mlp"
nonlinear_B_true = utils.simulate_dag(d, s0, graph_type)
nonlinear_dataset = utils.simulate_nonlinear_sem(nonlinear_B_true, n, sem_type)

# Using flex to implement NOTEARS

## linear NOTEARS

In [12]:
# Linear model
model = flex.LinearModel(d)

# Use AML to solve the constrained problem
cons_solver = flex.AugmentedLagrangian(
    num_iter=10,
    num_steps=[3e4,6e4],
    l1_coeff=0.03,
)

# Use Adam to solve the unconstrained problem
uncons_solver = flex.GradientBasedSolver(
    optimizer=torch.optim.Adam(model.parameters(), lr=3e-4),
)

# Use MSE loss
loss_fn = flex.MSELoss()

# Use Trace of matrix exponential as DAG function
dag_fn = flex.Exp()

# Learn the DAG
W_est = flex.struct_learn(
    dataset=linear_dataset,
    model=model,
    constrained_solver=cons_solver,
    unconstrained_solver=uncons_solver,
    loss_fn=loss_fn,
    dag_fn=dag_fn,
    w_threshold=0.3,
)

acc = utils.count_accuracy(linear_B_true, W_est != 0)
print("Results: ", acc)

100%|██████████| 10/10 [00:37<00:00,  3.79s/it]

Total Time: 37.8957781791687
Results:  {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 20}


## nonlinear NOTEARS

In [34]:
def dcdi_aug_lagrangian(n, d, s0, num_layers=2, noise_type="gauss"):
    graph_type, sem_type = "ER", "mlp"
    nonlinear_B_true = utils.simulate_dag(d, s0, graph_type)
    nonlinear_dataset = utils.simulate_nonlinear_sem(nonlinear_B_true, n, sem_type, noise_type=noise_type)

    # Nonlinear model
    model = flex.MLP(dims=[d, 1, d], num_layers=num_layers, hid_dim=16, activation="relu", bias=True)

    # Use AML to solve the constrained problem
    cons_solver = flex.AugmentedLagrangian(
        num_iter=1000000,
        num_steps=[1,1],
        l1_coeff=0.1,
        # weight_decay=0.01,
        rho_init=1e-8,
        rho_scale=2,
    )

    # Use Adam to solve the unconstrained problem
    uncons_solver = flex.GradientBasedSolver(
        optimizer=torch.optim.RMSprop(model.parameters(), lr=1e-3),
    )

    # Use MSE loss
    loss_fn = flex.MSELoss()

    dag_fn = flex.DCDI_h()

    # Learn the DAG
    W_est = flex.struct_learn(
        dataset=nonlinear_dataset,
        model=model,
        constrained_solver=cons_solver,
        unconstrained_solver=uncons_solver,
        loss_fn=loss_fn,
        dag_fn=dag_fn,
        w_threshold=0.0,
    )

    # W_est = postprocess(W_est)

    acc = utils.count_accuracy(nonlinear_B_true, W_est != 0)
    print("Results: ", acc)
    return acc

In [41]:
dcdi_aug_lagrangian(1000, 5, 5, num_layers=2, noise_type="gauss")

Total Time: 144.20864176750183
struct learn returning [[0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Results:  {'fdr': 0.5, 'tpr': 0.4, 'fpr': 0.4, 'shd': 4, 'sid': 7.0, 'nnz': 4}


{'fdr': 0.5, 'tpr': 0.4, 'fpr': 0.4, 'shd': 4, 'sid': 7.0, 'nnz': 4}

In [ ]:
shds = []
for i in range(10):
    acc = dcdi_aug_lagrangian(1000, 5, 5, num_layers=2, noise_type="gauss")
    shds.append(acc['shd'])
print("Average SHD: ", np.mean(shds))

# Using flex to implement DAGMA

## Linear DAGMA

In [14]:
# Linear model
model = flex.LinearModel(d)

# Use path following to solve the constrained problem
cons_solver = flex.PathFollowing(
    num_iter=5,
    mu_init=1.0,
    mu_scale=0.1,
    logdet_coeff=[1.0, .9, .8, .7, .6],
    num_steps=[3e4, 6e4],
    l1_coeff=0.03,
)

# use Adam to solve the unconstrained problem
uncons_solver = flex.GradientBasedSolver(
    optimizer=torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.99, 0.999))
)

# Use MSE loss
loss_fn = flex.MSELoss()

# Use LogDet as DAG function
dag_fn = flex.LogDet()

W_est = flex.struct_learn(
    dataset=linear_dataset,
    model=model,
    constrained_solver=cons_solver,
    unconstrained_solver=uncons_solver,
    loss_fn=loss_fn,
    dag_fn=dag_fn,
    w_threshold=0.3,
)

acc = utils.count_accuracy(linear_B_true, W_est != 0)
print("Results: ", acc)

/Users/seren/dagrad/dagrad/flex/modules/constrained_solvers.py:49: UserWarning: Using the first value from num_steps for the first 3 iterations
  self.vwarn(
100%|██████████| 5/5 [00:08<00:00,  1.77s/it]

Total Time: 8.867684841156006
Results:  {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 20}


## Nonlinear DAGMA

In [48]:
n, d, s0, graph_type, sem_type = 1000, 5, 5, "ER", "mlp"
nonlinear_B_true = utils.simulate_dag(d, s0, graph_type)
nonlinear_dataset = utils.simulate_nonlinear_sem(nonlinear_B_true, n, sem_type)

# nonlinear model
model = flex.MLP(dims=[d, 2, 5], num_layers=2, hid_dim=10, activation="sigmoid", bias=True)

# Use path following to solve the constrained problem
cons_solver = flex.PathFollowing(
    num_iter=4,
    mu_init=0.1,
    mu_scale=0.1,
    logdet_coeff=1.0,
    num_steps=[5e4, 8e4],
    weight_decay=0.02,
    l1_coeff=0.005,
)

# use Adam to solve the unconstrained problem
uncons_solver = flex.GradientBasedSolver(
    # optimizer=torch.optim.Adam(model.parameters(), lr=2e-4, betas=(0.99, 0.999))
    optimizer=torch.optim.SGD(model.parameters(), lr=2e-4)
)

# Use NLL loss
loss_fn = flex.NLLLoss()

# Use LogDet as DAG function
dag_fn = flex.LogDet()

# Learn the DAG
W_est = flex.struct_learn(
    dataset=nonlinear_dataset,
    model=model,
    constrained_solver=cons_solver,
    unconstrained_solver=uncons_solver,
    loss_fn=loss_fn,
    dag_fn=dag_fn,
    w_threshold=0.3,
)

W_est = postprocess(W_est)
acc = utils.count_accuracy(nonlinear_B_true, W_est != 0)
print("Results: ", acc)

/Users/seren/dagrad/dagrad/flex/modules/constrained_solvers.py:51: UserWarning: Using the first value from num_steps for the first 2 iterations
  self.vwarn(
100%|██████████| 4/4 [09:40<00:00, 145.12s/it]

Total Time: 580.489175081253
Results:  {'fdr': 0.0, 'tpr': 0.0, 'fpr': 0.0, 'shd': 5, 'nnz': 0}
